In [ ]:
<!DOCTYPE html>
<html lang="fr">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>Historique des Prix - {{ product.name }}</title>
    <link href="https://cdn.jsdelivr.net/npm/bootstrap@5.1.3/dist/css/bootstrap.min.css" rel="stylesheet">
    <script src="https://cdn.jsdelivr.net/npm/chart.js"></script>
    <style>
        .store-amazon { background-color: #ff9900; color: white; }
        .store-apple-store { background-color: #007aff; color: white; }
        .store-fnac { background-color: #e1000f; color: white; }
        .store-darty { background-color: #ff6600; color: white; }
        .store-boulanger { background-color: #0066cc; color: white; }
        .store-cdiscount { background-color: #ff6600; color: white; }
        .store-rue-du-commerce { background-color: #009639; color: white; }
        .store-ldlc { background-color: #0066cc; color: white; }
        .store-materiel-net { background-color: #ff6600; color: white; }
        .store-topachat { background-color: #ff0000; color: white; }
        .store-default { background-color: #6c757d; color: white; }

        .price-card {
            border-left: 4px solid #007bff;
            background: linear-gradient(135deg, #f8f9fa 0%, #e9ecef 100%);
        }

        .chart-container {
            position: relative;
            height: 400px;
            margin: 20px 0;
        }

        .store-badge {
            font-size: 0.8em;
            padding: 0.3em 0.6em;
            border-radius: 0.25rem;
            font-weight: 500;
        }

        .price-trend-up {
            color: #dc3545;
        }

        .price-trend-down {
            color: #28a745;
        }

        .price-trend-stable {
            color: #6c757d;
        }
    </style>
</head>
<body>
    <div class="container-fluid">
        <!-- Breadcrumb -->
        <nav aria-label="breadcrumb" class="mt-3">
            <ol class="breadcrumb">
                <li class="breadcrumb-item"><a href="/">Accueil</a></li>
                <li class="breadcrumb-item"><a href="/products">Produits</a></li>
                <li class="breadcrumb-item"><a href="/product/{{ product.id }}">{{ product.name }}</a></li>
                <li class="breadcrumb-item active" aria-current="page">Historique des Prix</li>
            </ol>
        </nav>

        <!-- En-tête du produit -->
        <div class="row mb-4">
            <div class="col-12">
                <h1 class="display-6">{{ product.name }}</h1>
                <p class="text-muted">{{ product.description or "Aucune description disponible" }}</p>
            </div>
        </div>

        <!-- Statistiques des prix -->
        <div class="row mb-4">
            <div class="col-md-3">
                <div class="card price-card">
                    <div class="card-body text-center">
                        <h5 class="card-title text-primary">Prix Actuel</h5>
                        <h3 class="text-dark">{{ "%.2f"|format(current_price) }} €</h3>
                        <small class="text-muted">Mis à jour: {{ last_update.strftime('%d/%m/%Y %H:%M') if last_update else 'N/A' }}</small>
                    </div>
                </div>
            </div>
            <div class="col-md-3">
                <div class="card price-card">
                    <div class="card-body text-center">
                        <h5 class="card-title text-success">Prix Min</h5>
                        <h3 class="text-dark">{{ "%.2f"|format(min_price) }} €</h3>
                        <small class="text-muted">{{ min_date.strftime('%d/%m/%Y') if min_date else 'N/A' }}</small>
                    </div>
                </div>
            </div>
            <div class="col-md-3">
                <div class="card price-card">
                    <div class="card-body text-center">
                        <h5 class="card-title text-danger">Prix Max</h5>
                        <h3 class="text-dark">{{ "%.2f"|format(max_price) }} €</h3>
                        <small class="text-muted">{{ max_date.strftime('%d/%m/%Y') if max_date else 'N/A' }}</small>
                    </div>
                </div>
            </div>
            <div class="col-md-3">
                <div class="card price-card">
                    <div class="card-body text-center">
                        <h5 class="card-title text-info">Prix Moyen</h5>
                        <h3 class="text-dark">{{ "%.2f"|format(avg_price) }} €</h3>
                        <small class="text-muted">Sur {{ total_entries }} relevés</small>
                    </div>
                </div>
            </div>
        </div>

        <!-- Graphique -->
        <div class="row mb-4">
            <div class="col-12">
                <div class="card">
                    <div class="card-header d-flex justify-content-between align-items-center">
                        <h5 class="mb-0">Évolution des Prix</h5>
                        <div>
                            <button class="btn btn-sm btn-outline-primary" onclick="changePeriod('7d')">7 jours</button>
                            <button class="btn btn-sm btn-outline-primary" onclick="changePeriod('30d')">30 jours</button>
                            <button class="btn btn-sm btn-outline-primary active" onclick="changePeriod('90d')">90 jours</button>
                            <button class="btn btn-sm btn-outline-primary" onclick="changePeriod('all')">Tout</button>
                        </div>
                    </div>
                    <div class="card-body">
                        <div class="chart-container">
                            <canvas id="priceChart"></canvas>
                        </div>
                        <div id="chartError" class="alert alert-warning d-none">
                            <i class="fas fa-exclamation-triangle"></i>
                            Erreur lors du chargement du graphique. Veuillez réessayer.
                        </div>
                    </div>
                </div>
            </div>
        </div>

        <!-- Tableau des prix -->
        <div class="row">
            <div class="col-12">
                <div class="card">
                    <div class="card-header d-flex justify-content-between align-items-center">
                        <h5 class="mb-0">Historique Détaillé</h5>
                        <div>
                            <a href="/product/{{ product.id }}/history/export" class="btn btn-sm btn-success">
                                <i class="fas fa-download"></i> Exporter CSV
                            </a>
                        </div>
                    </div>
                    <div class="card-body p-0">
                        <div class="table-responsive">
                            <table class="table table-striped table-hover mb-0">
                                <thead class="table-dark">
                                    <tr>
                                        <th>Date</th>
                                        <th>Prix</th>
                                        <th>Boutique</th>
                                        <th>Évolution</th>
                                        <th>URL</th>
                                    </tr>
                                </thead>
                                <tbody>
                                    {% for entry in price_history %}
                                    <tr>
                                        <td>{{ entry.checked_at.strftime('%d/%m/%Y %H:%M') }}</td>
                                        <td>
                                            <strong>{{ "%.2f"|format(entry.price) }} €</strong>
                                        </td>
                                        <td>
                                            <span class="store-badge store-{{ entry.store.name.lower().replace(' ', '-').replace('.', '') if entry.store else 'default' }}">
                                                {{ entry.store.name if entry.store else 'Inconnu' }}
                                            </span>
                                        </td>
                                        <td>
                                            {% if loop.index < price_history|length %}
                                                {% set prev_price = price_history[loop.index].price %}
                                                {% set diff = entry.price - prev_price %}
                                                {% if diff > 0 %}
                                                    <span class="price-trend-up">
                                                        <i class="fas fa-arrow-up"></i> +{{ "%.2f"|format(diff) }} €
                                                    </span>
                                                {% elif diff < 0 %}
                                                    <span class="price-trend-down">
                                                        <i class="fas fa-arrow-down"></i> {{ "%.2f"|format(diff) }} €
                                                    </span>
                                                {% else %}
                                                    <span class="price-trend-stable">
                                                        <i class="fas fa-minus"></i> Stable
                                                    </span>
                                                {% endif %}
                                            {% else %}
                                                <span class="text-muted">-</span>
                                            {% endif %}
                                        </td>
                                        <td>